# Band10 $CO_2$

```
!## band 10 (4400 - 8200)                       
real, parameter    :: vstar = 4400.0                     
integer, parameter :: nband = 380  
integer, parameter :: nv = 10000   
real, parameter    :: dv = 0.001 

integer   flgh2o, flgco2, flgo3, flgo2   
data      flgh2o, flgco2, flgo3, flgo2  
        /   0,      1,      0,      0 /  
        
parameter (cosz=1,rsfc=0.0)  
integer, parameter :: nref = 2  
integer, parameter :: max_ng = 15  
integer, parameter :: ng = 8  
integer, parameter :: nlayer = 75  
include 'atmosphere_profiles/saw75.pro'  
real, dimension(nref), parameter :: p_refs =(/ 10., 300. /)  !## h2o Band11 & co2 Band10  
real, dimension(nref), parameter :: t_refs =(/ 250., 250. /) !## h2o Band9,11 & co2 Band10,11  
integer, dimension(nref), parameter :: ng_refs = (/4, 4/)  
integer, dimension(nref), parameter :: ng_adju = (/ 0, 0 /)  !## co2 Band10,11   
data wgt  /.85, .85, .8, .8, .75, .7, .7, .85/  
integer, parameter :: option_klin = 1  
integer, parameter :: option_k_lookup = 0 ! 1-interpolation with ktable   
integer, parameter :: nl = 0   
integer, parameter :: nt = 0

clayer = 0.000400                      ! co2 (units:pppv)  
```


In [54]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from IPython import display
from pathlib import *
import pandas as pd
import xarray as xr
from bokeh.palettes import all_palettes
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
from climatools.plot.plot import *
output_notebook()

Loading BokehJS ...

In [36]:
! ls ../crdnew-sw/band10_-_co2_0.0004/

err_427256  fort.20   fort.402	lblcom.f	     out_427256
fort.10     fort.400  fort.403	lblnew-bestfit-sw.f
fort.19     fort.401  fort.9	lblnew-sw.exe


In [37]:
PATH = Path('../crdnew-sw/band10_-_co2_0.0004/')

In [38]:
np.random.seed(30)

## Line-by-line heating rates for each $g$

In [76]:
heat = pd.read_csv(PATH/'fort.401', header=None, sep=r'\s+')
heat.set_index(0, inplace=True)
heat.index.names = ['pressure']
heat.columns.names = ['g']

pltdata = [{'srs':xr.DataArray(heatg[g]),
            'label':f'{g}',
            'color':np.random.choice(all_palettes['Set1'][9]),
            'alpha': np.random.uniform(.5, 1.),
            'line_width':np.random.uniform(1, 5),
            'line_dash':np.random.choice(['solid', 'dashed', 'dotted', 'dotdash', 'dashdot'])} for g in heatg]

fig_linp = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='heat')
fig_logp = plt_vert_profile_bokeh(pltdata=pltdata, y_axis_type='log', xlabel='heat')
show(gridplot(fig_linp, fig_logp, ncols=2))

## k-dist heating rates for each $g$

In [78]:
heatg = pd.read_csv(PATH/'fort.400', header=None, sep=r'\s+')
heatg.set_index(0, inplace=True)
heatg.index.names = ['pressure']
heatg.columns.names = ['g']

pltdata = [{'srs':xr.DataArray(heatg[g]),
            'label':f'{g}',
            'color':np.random.choice(all_palettes['Set1'][9]),
            'alpha':np.random.uniform(.5, 1.),
            'line_width':np.random.uniform(1, 5),
            'line_dash':np.random.choice(['solid', 'dashed', 'dotted', 'dotdash', 'dashdot'])} for g in heatg]

fig_linp = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='heatg')
fig_logp = plt_vert_profile_bokeh(pltdata=pltdata, y_axis_type='log', xlabel='heatg')
show(gridplot(fig_linp, fig_logp, ncols=2))

## Heating rate (Line-by-line vs k-dist )

In [80]:
heatg = pd.read_csv(PATH/'fort.400', header=None, sep=r'\s+')
heatg.set_index(0, inplace=True)
heatg.index.names = ['pressure']
heatg.columns.names = ['g']

heat = pd.read_csv(PATH/'fort.401', header=None, sep=r'\s+')
heat.set_index(0, inplace=True)
heat.index.names = ['pressure']
heat.columns.names = ['g']

for g in heat.columns:
    pltdata = [{'srs':xr.DataArray(heatg[g]),
                'label':f'KDIS. g={g}',
                'color':all_palettes['Set1'][9][0],
                'alpha':.5,
                'line_width':5,
                'line_dash':'dotdash'},
               {'srs':xr.DataArray(heat[g]),
                'label':f'CRD. g={g}',
                'color':all_palettes['Set1'][9][1],
                'alpha':1,
                'line_width':2,
                'line_dash':'solid'}]

    fig_liny = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='Heating Rate', y_axis_type='linear', prange=(50, 1050))
    fig_logy = plt_vert_profile_bokeh(pltdata=pltdata, xlabel='Heating Rate', y_axis_type='log', prange=(1e-2, 200))
    show(gridplot(fig_liny, fig_logy, ncols=2))

## Bands-total fluxes

**tflux** -- line-by-line  
**tfluxg** -- k-dist

In [48]:
flux = pd.read_csv(PATH/'fort.9', sep=r'\s+', skiprows=2, header=None)
flux.set_index(0, inplace=True)
flux.index.names = ['pressure']
flux.columns = ['tflux', 'tfluxg']

pltdata = [{'srs':xr.DataArray(flux[m]),
            'label':f'{m}',
            'color':np.random.choice(all_palettes['Category20'][20]),
            'alpha':np.random.uniform(.5, 1.),
            'line_width':np.random.uniform(1, 5),
            'line_dash':np.random.choice(['solid', 'dashed', 'dotted', 'dotdash', 'dashdot'])} for m in flux]

show(plt_vert_profile_bokeh(pltdata=pltdata, xlabel='flux'))

## Bands-total heating 

**theat** -- line-by-line  
**theatg** -- k-dist

In [65]:
heat = pd.read_csv(PATH/'fort.10', sep=r'\s+', header=None)
heat.set_index(0, inplace=True)
heat.index.names = ['pressure']
heat.columns = ['theat', 'theatg']

pltdata = [{'srs':xr.DataArray(heat[m]),
            'label':f'{m}',
            'color':np.random.choice(all_palettes['Category20'][20]),
            'alpha':np.random.uniform(.5, 1.),
            'line_width':np.random.uniform(1, 5),
            'line_dash':np.random.choice(['solid', 'dashed', 'dotted', 'dotdash', 'dashdot'])} for m in heat]

show(plt_vert_profile_bokeh(pltdata=pltdata, xlabel='heat'))

# fin

In [66]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')